# List of Exposures with Hologram at USDF


- work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- last update : 2023/08/30



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_27/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/ctrl_mpexec/g88183d3282+88d805ed8c/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/spectractor/g7bf34a7a6e/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/obs_lsst/gfd7e96728c+6f6aa72e7b/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/pipe_tasks/gfb267207ab+bb26f3254a/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/shapelet/gd4112516d0+5ce2cc5508/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/meas_modelfit/g807e3ba568+b7d80ac9cf/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/scarlet_extensions/g9d18589735+d445b4966c/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/proxmin/g33b4157f25/l

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_27/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/eups 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

Weekly 2023_27
   gdf42428520+4e9f6d16c8 	current w_2023_27 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [12]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

exposure: 
  instrument: str
  id: int
  physical_filter: str
  obs_id: str
  exposure_time: float
  dark_time: float
  observation_type: str
  observation_reason: str
  day_obs: int
  seq_num: int
  seq_start: int
  seq_end: int
  group_name: str
  group_id: int
  target_name: str
  science_program: str
  tracking_ra: float
  tracking_dec: float
  sky_angle: float
  azimuth: float
  zenith_angle: float
  has_simulated: bool
  timespan: lsst.daf.butler.Timespan


In [13]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [14]:
info.toDict()

{'instrument': 'LATISS',
 'id': 2023082900581,
 'physical_filter': 'empty~blue300lpmm_qn1',
 'obs_id': 'AT_O_20230829_000581',
 'exposure_time': 40.0,
 'dark_time': 40.2345,
 'observation_type': 'science',
 'observation_reason': 'generic',
 'day_obs': 20230829,
 'seq_num': 581,
 'seq_start': 581,
 'seq_end': 581,
 'group_name': '2023-08-30T05:51:05.449',
 'group_id': 2733042654490000,
 'target_name': 'HD185975',
 'science_program': 'spec',
 'tracking_ra': 307.77748055172003,
 'tracking_dec': -87.50362169767645,
 'sky_angle': 133.11323897406663,
 'azimuth': 182.15453619976546,
 'zenith_angle': 57.93913032481956,
 'has_simulated': False,
 'timespan': Timespan(begin=astropy.time.Time('2023-08-30 05:53:44.111997', scale='tai', format='iso'), end=astropy.time.Time('2023-08-30 05:54:24.347000', scale='tai', format='iso'))}

In [15]:
info

exposure.RecordClass(instrument='LATISS', id=2023082900581, physical_filter='empty~blue300lpmm_qn1', obs_id='AT_O_20230829_000581', exposure_time=40.0, dark_time=40.2345, observation_type='science', observation_reason='generic', day_obs=20230829, seq_num=581, seq_start=581, seq_end=581, group_name='2023-08-30T05:51:05.449', group_id=2733042654490000, target_name='HD185975', science_program='spec', tracking_ra=307.77748055172003, tracking_dec=-87.50362169767645, sky_angle=133.11323897406663, azimuth=182.15453619976546, zenith_angle=57.93913032481956, has_simulated=False, timespan=Timespan(begin=astropy.time.Time('2023-08-30 05:53:44.111997', scale='tai', format='iso'), end=astropy.time.Time('2023-08-30 05:54:24.347000', scale='tai', format='iso')))

In [16]:
info.azimuth

182.15453619976546

In [17]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57069,2023082900569,AT_O_20230829_000569,20230829,569,2023-08-30 05:45:41.807994,2023-08-30 05:45:43.038,acq,HD14943,SDSSr_65mm~blue300lpmm_qn1,37.445130,1.0,35.752486,-51.049914,15.738998,135.796577,37.445130,spec,2.460187e+06,60186.240067
57070,2023082900572,AT_O_20230829_000572,20230829,572,2023-08-30 05:47:56.661015,2023-08-30 05:48:01.899,acq,HD185975,empty~holo4_003,57.894137,5.0,307.077558,-87.472194,138.113914,182.152529,57.894137,spec,2.460187e+06,60186.241628
57071,2023082900575,AT_O_20230829_000575,20230829,575,2023-08-30 05:49:36.253997,2023-08-30 05:50:06.506,science,HD185975,empty~holo4_003,57.909176,30.0,307.718101,-87.505801,138.116280,182.116371,57.909176,spec,2.460187e+06,60186.242781
57072,2023082900578,AT_O_20230829_000578,20230829,578,2023-08-30 05:51:35.818989,2023-08-30 05:51:41.054,acq,HD185975,empty~holo4_003,57.921377,5.0,307.786904,-87.503362,133.103629,182.134758,57.921377,spec,2.460187e+06,60186.244165


In [18]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [19]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57069,2023082900569,AT_O_20230829_000569,20230829,569,2023-08-30 05:45:41.807994,2023-08-30 05:45:43.038,acq,HD14943,SDSSr_65mm~blue300lpmm_qn1,37.445130,1.0,35.752486,-51.049914,15.738998,135.796577,37.445130,spec,2.460187e+06,60186.240067
57070,2023082900572,AT_O_20230829_000572,20230829,572,2023-08-30 05:47:56.661015,2023-08-30 05:48:01.899,acq,HD185975,empty~holo4_003,57.894137,5.0,307.077558,-87.472194,138.113914,182.152529,57.894137,spec,2.460187e+06,60186.241628
57071,2023082900575,AT_O_20230829_000575,20230829,575,2023-08-30 05:49:36.253997,2023-08-30 05:50:06.506,science,HD185975,empty~holo4_003,57.909176,30.0,307.718101,-87.505801,138.116280,182.116371,57.909176,spec,2.460187e+06,60186.242781
57072,2023082900578,AT_O_20230829_000578,20230829,578,2023-08-30 05:51:35.818989,2023-08-30 05:51:41.054,acq,HD185975,empty~holo4_003,57.921377,5.0,307.786904,-87.503362,133.103629,182.134758,57.921377,spec,2.460187e+06,60186.244165


## Selection of science

In [20]:
df_science = df_exposure[df_exposure.type == 'science']

In [21]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_6913/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [22]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
56976,2023082900485,AT_O_20230829_000485,20230829,485,2023-08-30 04:43:52.888985,2023-08-30 04:44:23.128,science,Photo1800-1_025,SDSSi_65mm~empty,59.312492,30.0,270.144150,-24.960376,359.988447,257.161795,59.312492,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.197140
56977,2023082900486,AT_O_20230829_000486,20230829,486,2023-08-30 04:44:45.170013,2023-08-30 04:45:15.400,science,Photo1800-1_026,SDSSi_65mm~empty,59.481635,30.0,270.160431,-24.960359,359.988361,257.083490,59.481635,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.197745
56978,2023082900487,AT_O_20230829_000487,20230829,487,2023-08-30 04:45:29.681017,2023-08-30 04:45:59.924,science,Photo1800-1_026,SDSSg_65mm~empty,59.639864,30.0,270.160352,-24.960357,359.988268,257.010137,59.639864,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.198260
56979,2023082900488,AT_O_20230829_000488,20230829,488,2023-08-30 04:46:14.053015,2023-08-30 04:46:44.296,science,Photo1800-1_026,SDSSr_65mm~empty,59.794506,30.0,270.160313,-24.960340,359.988191,256.938428,59.794506,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.198774
56980,2023082900489,AT_O_20230829_000489,20230829,489,2023-08-30 04:47:06.309018,2023-08-30 04:47:36.549,science,Photo1800-1_027,SDSSr_65mm~empty,59.963364,30.0,270.176728,-24.960345,359.987914,256.860098,59.963364,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.199379
56981,2023082900490,AT_O_20230829_000490,20230829,490,2023-08-30 04:47:50.819981,2023-08-30 04:48:21.045,science,Photo1800-1_027,SDSSg_65mm~empty,60.121407,30.0,270.176694,-24.960342,359.987821,256.786707,60.121407,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.199894
56982,2023082900491,AT_O_20230829_000491,20230829,491,2023-08-30 04:48:35.245007,2023-08-30 04:49:05.482,science,Photo1800-1_027,SDSSi_65mm~empty,60.275844,30.0,270.176704,-24.960362,359.987520,256.714922,60.275844,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.200408
56983,2023082900492,AT_O_20230829_000492,20230829,492,2023-08-30 04:49:26.569009,2023-08-30 04:49:56.809,science,Photo1800-1_029,SDSSi_65mm~empty,60.430901,30.0,270.209405,-24.960331,359.987372,256.642965,60.430901,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.201002
56984,2023082900493,AT_O_20230829_000493,20230829,493,2023-08-30 04:50:11.009002,2023-08-30 04:50:41.248,science,Photo1800-1_029,SDSSg_65mm~empty,60.585279,30.0,270.209371,-24.960374,359.987224,256.571097,60.585279,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.201516
56985,2023082900494,AT_O_20230829_000494,20230829,494,2023-08-30 04:50:55.331996,2023-08-30 04:51:25.568,science,Photo1800-1_029,SDSSr_65mm~empty,60.739644,30.0,270.209335,-24.960333,359.987080,256.499295,60.739644,AUXTEL_PHOTO_IMAGING,2.460187e+06,60186.202029


In [23]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706, 20230718,
       20230720, 20230801, 20230802, 20230803, 20230815, 20230816,
       20230817, 20230829])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [24]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
#date_sel= 20230118
#date_sel= 20230131

# data in February
#date_sel = 20230201
#date_sel = 20230202
#date_sel = 20230704

#date_sel = 20230510
#date_sel = 20230509
#date_sel = 20230511

#date_sel = 20230705
#date_sel = 20230706

#july 2023
#date_sel = 20230718
#date_sel = 20230720

#august 2023
date_sel = 20230801
#date_sel = 20230802

#date_sel = 20230815
#date_sel = 20230817



## Query Dimension records on Exposures

In [25]:
where_clause = f"instrument=\'LATISS\' AND exposure.day_obs={date_sel} AND exposure.science_program=\'spec\'"

In [26]:
results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = 'LATISS/raw/all',where=where_clause)

## Select date

In [27]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [28]:
len(df_science_selected)

140

# List of filters

In [29]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003', 'empty~blue300lpmm_qn1', 'empty~empty',
       'SDSSr_65mm~empty', 'SDSSg_65mm~empty', 'SDSSi_65mm~empty'],
      dtype=object)

In [30]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [31]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [32]:
pd.set_option('display.max_rows', None)

In [33]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20230801_filt_empty-holo4_003.list


/tmp/ipykernel_6913/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [34]:
N = len(all_def_science_selected_final)

In [35]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
53113,2023080100112,AT_O_20230801_000112,20230801,112,2023-08-01 23:17:43.449003,2023-08-01 23:18:13.684,science,HD185975,empty~holo4_003,59.421445,30.0,308.443539,-87.434731,9.522152,176.951992,59.421445,spec,2.460158e+06,60157.970642
53109,2023080100113,AT_O_20230801_000113,20230801,113,2023-08-01 23:18:26.539012,2023-08-01 23:18:56.775,science,HD185975,empty~holo4_003,59.413222,30.0,308.443946,-87.434743,9.521665,176.953030,59.413222,spec,2.460158e+06,60157.971140
53110,2023080100116,AT_O_20230801_000116,20230801,116,2023-08-01 23:20:18.271981,2023-08-01 23:20:48.524,science,HD185975,empty~holo4_003,59.387008,30.0,308.358376,-87.428601,4.519842,176.949508,59.387008,spec,2.460158e+06,60157.972434
53112,2023080100117,AT_O_20230801_000117,20230801,117,2023-08-01 23:21:01.275007,2023-08-01 23:21:31.517,science,HD185975,empty~holo4_003,59.378751,30.0,308.358202,-87.428618,4.519922,176.950684,59.378751,spec,2.460158e+06,60157.972931
53417,2023080100159,AT_O_20230801_000159,20230801,159,2023-08-02 03:55:31.833990,2023-08-02 03:56:02.063,science,HD146233,empty~holo4_003,55.011272,30.0,243.878072,-8.409854,211.278042,281.525617,55.011272,spec,2.460159e+06,60158.163563
53418,2023080100160,AT_O_20230801_000160,20230801,160,2023-08-02 03:56:15.045986,2023-08-02 03:56:45.288,science,HD146233,empty~holo4_003,55.163310,30.0,243.878058,-8.409821,211.278059,281.413623,55.163310,spec,2.460159e+06,60158.164063
53402,2023080100163,AT_O_20230801_000163,20230801,163,2023-08-02 03:58:06.603016,2023-08-02 03:58:36.843,science,HD146233,empty~holo4_003,55.551853,30.0,243.881482,-8.412527,206.275901,281.125150,55.551853,spec,2.460159e+06,60158.165354
53403,2023080100164,AT_O_20230801_000164,20230801,164,2023-08-02 03:58:49.656011,2023-08-02 03:59:19.907,science,HD146233,empty~holo4_003,55.704132,30.0,243.881433,-8.412491,206.275956,281.014295,55.704132,spec,2.460159e+06,60158.165853
53613,2023080100173,AT_O_20230801_000173,20230801,173,2023-08-02 04:04:07.105989,2023-08-02 04:04:37.344,science,HD146233,empty~holo4_003,56.835561,30.0,243.878114,-8.410188,210.788444,280.204386,56.835561,spec,2.460159e+06,60158.169527
53510,2023080100174,AT_O_20230801_000174,20230801,174,2023-08-02 04:04:50.095013,2023-08-02 04:05:20.333,science,HD146233,empty~holo4_003,56.988259,30.0,243.878092,-8.410176,210.788513,280.096081,56.988259,spec,2.460159e+06,60158.170024


In [36]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

In [37]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

In [38]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [39]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [40]:
df_forsave = pd.concat(all_def_science_selected_final)

In [41]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [42]:
filename_out_csv

'holosummary_20230801.csv'

In [43]:
df_forsave.to_csv(filename_out_csv) 

In [44]:
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [45]:
df_forsave.to_excel(filename_out_excel) 